In [3]:
# ==========================
# Ví dụ Transformer đơn giản
# ==========================

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Transformer
import math

# 1️⃣ Cấu hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SRC_VOCAB_SIZE = 100  # từ vựng đầu vào
TGT_VOCAB_SIZE = 100  # từ vựng đầu ra
EMB_SIZE = 64         # kích thước vector nhúng
NHEAD = 4             # số "head" trong multi-head attention
FFN_HID_DIM = 128     # kích thước tầng feed-forward
BATCH_SIZE = 8
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

# 2️⃣ Định nghĩa mô hình Transformer
class TransformerModel(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, emb_size, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward):
        super().__init__()
        self.src_emb = nn.Embedding(src_vocab, emb_size)
        self.tgt_emb = nn.Embedding(tgt_vocab, emb_size)
        self.pos_encoder = PositionalEncoding(emb_size)
        self.transformer = Transformer(
            d_model=emb_size,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward
        )
        self.fc_out = nn.Linear(emb_size, tgt_vocab)

    def forward(self, src, tgt):
        src_emb = self.pos_encoder(self.src_emb(src))
        tgt_emb = self.pos_encoder(self.tgt_emb(tgt))
        output = self.transformer(src_emb, tgt_emb)
        return self.fc_out(output)

# 3️⃣ Positional Encoding (thêm thông tin vị trí cho từ)
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size, maxlen=5000):
        super().__init__()
        pe = torch.zeros(maxlen, emb_size)
        position = torch.arange(0, maxlen, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, emb_size, 2).float() * (-math.log(10000.0) / emb_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

# 4️⃣ Dữ liệu giả lập (batch gồm 8 câu, mỗi câu có 10 token)
src = torch.randint(0, SRC_VOCAB_SIZE, (10, BATCH_SIZE)).to(device)
tgt = torch.randint(0, TGT_VOCAB_SIZE, (10, BATCH_SIZE)).to(device)
tgt_y = torch.randint(0, TGT_VOCAB_SIZE, (10, BATCH_SIZE)).to(device)

# 5️⃣ Huấn luyện ngắn minh họa
model = TransformerModel(SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, EMB_SIZE, NHEAD, NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, FFN_HID_DIM).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(3):
    optimizer.zero_grad()
    output = model(src, tgt)
    loss = criterion(output.view(-1, TGT_VOCAB_SIZE), tgt_y.view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 4.7296
Epoch 2, Loss: 4.4947
Epoch 3, Loss: 4.3028
